In [1]:
import os
import zarr
import random
import json
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch
import torchvision.transforms.functional as F
import random
import sys
from collections import defaultdict

warnings.filterwarnings("ignore")
sys.path.append("./src/")

from src.config import CFG
from src.dataloader import (
    read_zarr,
    read_info_json,
    scale_coordinates,
    create_dataset,
    create_segmentation_map,
    EziiDataset,
    drop_padding,
)
from src.network import UNet_2D, aug
from src.utils import save_images
from src.metric import score, create_cls_pos, create_cls_pos_sikii, create_df

sample_submission = pd.read_csv("../../inputs/sample_submission.csv")

In [2]:
train_dataset = EziiDataset(
    exp_names=CFG.train_exp_names,
    base_dir="../../inputs/train",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.train_zarr_types,
    train=True,
)

valid_dataset = EziiDataset(
    exp_names=CFG.valid_exp_names,
    # exp_names=CFG.train_exp_names,
    base_dir="../../inputs/train",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.valid_zarr_types,
    train=True,
)

from tqdm import tqdm

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False)

for row in tqdm(valid_loader):
    normalized_tomogram = row["normalized_tomogram"]
    break

[('TS_5_4', 'denoised'), ('TS_5_4', 'ctfdeconvolved'), ('TS_5_4', 'wbp'), ('TS_5_4', 'isonetcorrected'), ('TS_73_6', 'denoised'), ('TS_73_6', 'ctfdeconvolved'), ('TS_73_6', 'wbp'), ('TS_73_6', 'isonetcorrected'), ('TS_99_9', 'denoised'), ('TS_99_9', 'ctfdeconvolved'), ('TS_99_9', 'wbp'), ('TS_99_9', 'isonetcorrected'), ('TS_6_4', 'denoised'), ('TS_6_4', 'ctfdeconvolved'), ('TS_6_4', 'wbp'), ('TS_6_4', 'isonetcorrected'), ('TS_69_2', 'denoised'), ('TS_69_2', 'ctfdeconvolved'), ('TS_69_2', 'wbp'), ('TS_69_2', 'isonetcorrected')]
[('TS_86_3', 'denoised'), ('TS_6_6', 'denoised')]


  0%|          | 0/2 [00:00<?, ?it/s]


In [3]:
class PadToSize(nn.Module):
    def __init__(self, resolution):
        super().__init__()
        if resolution == "0":
            self.size = 640
        elif resolution == "1":
            self.size = 320
        elif resolution == "2":
            self.size = 160

    def forward(self, x):
        return F.pad(x, (0, 0, self.size - x.shape[-1], self.size - x.shape[-2]))

In [4]:
model = UNet_2D().to("cuda")
model.eval()
model.load_state_dict(torch.load("./best_model.pth"))


optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(
    weight=torch.tensor([0.5, 32, 32, 32, 32, 32, 32]).to("cuda")
)
# criterion = DiceLoss()

best_model = None
best_loss = np.inf
batch_size = 4

valid_loss = []
valid_pred_tomogram = defaultdict(list)
valid_gt_tomogram = defaultdict(list)
model.eval()
tq = tqdm(range(len(valid_loader) * normalized_tomogram.shape[0]))
for data in valid_loader:
    exp_name = data["exp_name"][0]
    tomogram = data["normalized_tomogram"].to("cuda")
    segmentation_map = data["segmentation_map"].to("cuda").long()

    for i in range(tomogram.shape[1]):
        input_ = tomogram[:, i].unsqueeze(0)
        gt = segmentation_map[:, i]

        input_ = PadToSize(CFG.resolution)(input_)
        gt = PadToSize(CFG.resolution)(gt)
        output = model(input_)
        output = nn.functional.softmax(output, dim=1)
        loss = criterion(output, gt)

        valid_loss.append(loss.item())
        tq.set_description(f"Loss: {np.mean(valid_loss)}")
        tq.update(1)

        output = drop_padding(output, CFG.resolution)

        valid_pred_tomogram[exp_name].append(output.cpu().detach().numpy())
        valid_gt_tomogram[exp_name].append(gt.cpu().detach().numpy())
tq.close()

Loss: 1.4293923941643343: : 184it [00:04, 42.35it/s]                   


In [5]:
def create_gt_df(base_dir, exp_names):
    result_df = None
    particle_names = CFG.particles_name

    for exp_name in exp_names:
        for particle in particle_names:
            np_corrds = read_info_json(
                base_dir=base_dir, exp_name=exp_name, particle_name=particle
            )  # (n, 3)
            # 各行にexp_nameとparticle_name追加
            particle_df = pd.DataFrame(np_corrds, columns=["z", "y", "x"])
            particle_df["experiment"] = exp_name
            particle_df["particle_type"] = particle

            if result_df is None:
                result_df = particle_df
            else:
                result_df = pd.concat([result_df, particle_df], axis=0).reset_index(
                    drop=True
                )

    result_df = result_df.reset_index()  # index	experiment	particle_type	x	y	z
    result_df = result_df[["index", "experiment", "particle_type", "x", "y", "z"]]

    return result_df

In [6]:
gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", CFG.valid_exp_names)
gt_df = gt_df[gt_df["particle_type"] != "beta-amylase"].reset_index(drop=True)

In [7]:
# pred_df = pd.read_csv("../../inputs/train_submission.csv")


def calc_score(initial_sikii):
    all_pred_df = None

    for exp_name in CFG.valid_exp_names:
        pred_tomogram = valid_pred_tomogram[exp_name]
        pred_tomogram = np.array(pred_tomogram)  # (92, 1, 7, 315, 315)
        pred_tomogram = pred_tomogram.squeeze(1)  # (92, 7, 315, 315)

        pred_cls_pos, pred_Ascale_pos = create_cls_pos_sikii(
            pred_tomogram, sikii_dict=initial_sikii
        )
        pred_df = create_df(pred_Ascale_pos, exp_name)
        # pred_df = create_df(pred_cls_pos, exp_name)

        if all_pred_df is None:
            all_pred_df = pred_df
        else:
            all_pred_df = pd.concat([all_pred_df, pred_df], axis=0).reset_index(
                drop=True
            )

    pred_df = all_pred_df[all_pred_df["particle_type"] != "beta-amylase"]
    pred_df = pred_df.drop_duplicates(subset=["x", "y", "z"], keep="first").reset_index(
        drop=True
    )

    pred_df = pred_df.reset_index()

    score_ = score(
        pred_df, gt_df, row_id_column_name="index", distance_multiplier=1, beta=4
    )

    return score_


def calc_score_by_exp(initial_sikii):
    exp_scores = {}

    for exp_name in CFG.valid_exp_names:
        gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", [exp_name])

        pred_tomogram = valid_pred_tomogram[exp_name]
        pred_tomogram = np.array(pred_tomogram)  # (92, 1, 7, 315, 315)
        pred_tomogram = pred_tomogram.squeeze(1)  # (92, 7, 315, 315)

        pred_cls_pos, pred_Ascale_pos = create_cls_pos_sikii(
            pred_tomogram, sikii_dict=initial_sikii
        )
        pred_df = create_df(pred_Ascale_pos, exp_name)

        pred_df = pred_df[pred_df["particle_type"] != "beta-amylase"]
        pred_df = pred_df.drop_duplicates(
            subset=["x", "y", "z"], keep="first"
        ).reset_index(drop=True)

        pred_df = pred_df.reset_index()

        score_ = score(
            pred_df, gt_df, row_id_column_name="index", distance_multiplier=1, beta=4
        )

        exp_scores[exp_name] = score_

    return exp_scores

In [8]:
constant = 0.35

initial_sikii = {
    "apo-ferritin": constant,
    "beta-amylase": constant,
    "beta-galactosidase": constant,
    "ribosome": constant,
    "thyroglobulin": constant,
    "virus-like-particle": constant,
}

score_ = calc_score(initial_sikii)
score_

0.36860293385709647

In [9]:
constant = 0.35

initial_sikii = {
    "apo-ferritin": constant,
    "beta-amylase": constant,
    "beta-galactosidase": constant,
    "ribosome": constant,
    "thyroglobulin": constant,
    "virus-like-particle": constant,
}

score_ = calc_score_by_exp(initial_sikii)
score_

{'TS_86_3': 0.40603102493658433, 'TS_6_6': 0.3223497699636737}

In [10]:
best_sikii = 0
best_score = -np.inf

for sikii in np.linspace(0.3, 0.7, 100):
    initial_sikii = {
        "apo-ferritin": sikii,
        "beta-amylase": sikii,
        "beta-galactosidase": sikii,
        "ribosome": sikii,
        "thyroglobulin": sikii,
        "virus-like-particle": sikii,
    }
    score_ = calc_score(initial_sikii)
    if score_ > best_score:
        best_score = score_
        best_sikii = sikii
    print(sikii, score_)

0.3 0.33673918066122355
0.30404040404040406 0.3411127229927137
0.30808080808080807 0.34304248125087844
0.31212121212121213 0.3495508827470091
0.31616161616161614 0.3509147590227553
0.3202020202020202 0.3543681500852413
0.3242424242424242 0.36157162044766933
0.3282828282828283 0.3599567445543362
0.3323232323232323 0.3587959760392257
0.33636363636363636 0.3595025750060075
0.3404040404040404 0.35896568883692886
0.34444444444444444 0.3628377477787102
0.34848484848484845 0.36446114363517923
0.3525252525252525 0.3700907454751164
0.35656565656565653 0.3755850867218819
0.3606060606060606 0.37808045109985955
0.36464646464646466 0.3804979305677422
0.3686868686868687 0.3766523772725905
0.3727272727272727 0.38082582661259096
0.37676767676767675 0.386678644108659
0.3808080808080808 0.38922909491571916
0.38484848484848483 0.3945733305202078
0.3888888888888889 0.39107533074233103
0.3929292929292929 0.39389194332909955
0.396969696969697 0.39527181345896967
0.401010101010101 0.39713256869768687
0.40505

In [11]:
best_sikii, best_score

(0.5666666666666667, 0.5293386277750814)

# Train-Dataset

In [12]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(
    weight=torch.tensor([0.5, 32, 32, 32, 32, 32, 32]).to("cuda")
)
# criterion = DiceLoss()

best_model = None
best_loss = np.inf
batch_size = 4

train_loss = []
valid_pred_tomogram = defaultdict(list)
valid_gt_tomogram = defaultdict(list)
model.eval()
tq = tqdm(range(len(train_loader) * normalized_tomogram.shape[0]))
for data in train_loader:
    exp_name = data["exp_name"][0]
    tomogram = data["normalized_tomogram"].to("cuda")
    segmentation_map = data["segmentation_map"].to("cuda").long()

    for i in range(tomogram.shape[1]):
        input_ = tomogram[:, i].unsqueeze(0)
        gt = segmentation_map[:, i]

        input_ = PadToSize(CFG.resolution)(input_)
        gt = PadToSize(CFG.resolution)(gt)
        output = model(input_)
        output = nn.functional.softmax(output, dim=1)
        loss = criterion(output, gt)

        train_loss.append(loss.item())
        tq.set_description(f"Loss: {np.mean(train_loss)}")
        tq.update(1)

        output = drop_padding(output, CFG.resolution)

        valid_pred_tomogram[exp_name].append(output.cpu().detach().numpy())
        valid_gt_tomogram[exp_name].append(gt.cpu().detach().numpy())
tq.close()

Loss: 1.4310990602425906: : 1840it [00:32, 57.34it/s]                    


In [13]:
def calc_score(initial_sikii):
    all_pred_df = None

    for exp_name in CFG.train_exp_names:
        pred_tomogram = valid_pred_tomogram[exp_name]
        pred_tomogram = np.array(pred_tomogram)  # (92, 1, 7, 315, 315)
        pred_tomogram = pred_tomogram.squeeze(1)  # (92, 7, 315, 315)

        pred_cls_pos, pred_Ascale_pos = create_cls_pos_sikii(
            pred_tomogram, sikii_dict=initial_sikii
        )
        pred_df = create_df(pred_Ascale_pos, exp_name)
        # pred_df = create_df(pred_cls_pos, exp_name)

        if all_pred_df is None:
            all_pred_df = pred_df
        else:
            all_pred_df = pd.concat([all_pred_df, pred_df], axis=0).reset_index(
                drop=True
            )

    pred_df = all_pred_df[all_pred_df["particle_type"] != "beta-amylase"]
    pred_df = pred_df.drop_duplicates(subset=["x", "y", "z"], keep="first").reset_index(
        drop=True
    )

    pred_df = pred_df.reset_index()

    score_ = score(
        pred_df, gt_df, row_id_column_name="index", distance_multiplier=1, beta=4
    )

    return score_

In [14]:
gt_df = create_gt_df("../../inputs/train/overlay/ExperimentRuns/", CFG.train_exp_names)
gt_df = gt_df[gt_df["particle_type"] != "beta-amylase"].reset_index(drop=True)

In [15]:
best_sikii = 0
best_score = -np.inf

for sikii in np.linspace(0.25, 0.45, 100):
    initial_sikii = {
        "apo-ferritin": sikii,
        "beta-amylase": sikii,
        "beta-galactosidase": sikii,
        "ribosome": sikii,
        "thyroglobulin": sikii,
        "virus-like-particle": sikii,
    }
    score_ = calc_score(initial_sikii)
    if score_ > best_score:
        best_score = score_
        best_sikii = sikii
    print(sikii, score_)

0.25 0.29132777065298665
0.25202020202020203 0.2927766880881479
0.25404040404040407 0.29350447569485416
0.25606060606060604 0.29442226483249534
0.2580808080808081 0.2952753951595782
0.2601010101010101 0.29641173006205646
0.26212121212121214 0.2973902797084916
0.2641414141414141 0.29877998142344675
0.26616161616161615 0.30005512468012313
0.2681818181818182 0.301765411711211
0.2702020202020202 0.3032512396286127
0.2722222222222222 0.30380683026735417
0.27424242424242423 0.3044897584741107
0.27626262626262627 0.30434947213184455
0.2782828282828283 0.30428094295533825
0.2803030303030303 0.30504629928258253
0.2823232323232323 0.304276383292793
0.28434343434343434 0.30519169595338547
0.2863636363636364 0.3058865574382293
0.2883838383838384 0.30765069660286104
0.2904040404040404 0.30818807174245794
0.2924242424242424 0.30890508698811026
0.29444444444444445 0.30922328928753157
0.2964646464646465 0.309424359829029
0.29848484848484846 0.31020475100621564
0.3005050505050505 0.31177813736226995
0.